In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/snli/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-beta1

In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import math

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-beta1
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
# stream data from text files
def data_generator(f_path, word2idx):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      text1 = [word2idx.get(w, len(word2idx)) for w in text1.split()]
      text2 = [word2idx.get(w, len(word2idx)) for w in text2.split()]
      yield ((text1, text2), label2idx[label])


def dataset(is_training, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.int32, tf.int32), tf.int32)
  _pads = ((0, 0), -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
class FFNBlock(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name = name)
    self.dropout1 = tf.keras.layers.Dropout(0.3)
    self.fc1 = tf.keras.layers.Dense(params['hidden_units'], tf.nn.elu)
    self.dropout2 = tf.keras.layers.Dropout(0.2)
    self.fc2 = tf.keras.layers.Dense(params['hidden_units'], tf.nn.elu)
  
  def call(self, inputs, training=False):
    x = inputs
    x = self.dropout1(x, training=training)
    x = self.fc1(x)
    x = self.dropout2(x, training=training)
    x = self.fc2(x)
    return x

In [0]:
class Pyramid(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    self.embedding = tf.Variable(np.load(params['embedding_path']), name='pretrained_embedding')
    
    self.inp_dropout = tf.keras.layers.Dropout(0.2)
    
    self.encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['hidden_units'], return_sequences=True), name='encoder')
    
    self.conv_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=7, activation=tf.nn.elu, padding='same')
    
    self.conv_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.elu, padding='same')
    
    self.conv_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.elu, padding='same')
    
    self.W_1_1 = tf.keras.layers.Dense(params['hidden_units'], use_bias=False)
    
    self.W_1_2 = tf.keras.layers.Dense(params['hidden_units'], use_bias=False)
    
    self.v_1 = tf.keras.layers.Dense(1, use_bias=False)
    
    self.W_2 = tf.keras.layers.Dense(params['hidden_units'], use_bias=False)
    
    self.v_2 = tf.keras.layers.Dense(1, use_bias=False)
    
    self.flatten = tf.keras.layers.Flatten()
    
    self.out_hidden = FFNBlock(params, name='out_hidden')
    
    self.out_linear = tf.keras.layers.Dense(params['num_labels'], name='out_linear')
  
  
  def call(self, inputs, training=False):
    x1, x2 = inputs
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    len1, len2 = x1.shape[1], x2.shape[1]
    stride1, stride2 = len1 // params['fixed_len1'], len2 // params['fixed_len2']
    
    if len1 // stride1 != params['fixed_len1']:
      remin = (stride1 + 1) * params['fixed_len1'] - len1
      zeros = tf.zeros([batch_sz, remin], tf.int32)
      x1 = tf.concat([x1, zeros], 1)
      len1 = x1.shape[1]
      stride1 = len1 // params['fixed_len1']
    
    if len2 // stride2 != params['fixed_len2']:
      remin = (stride2 + 1) * params['fixed_len2'] - len2
      zeros = tf.zeros([batch_sz, remin], tf.int32)
      x2 = tf.concat([x2, zeros], 1)
      len2 = x2.shape[1]
      stride2 = len2 // params['fixed_len2']
    
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    
    mask1 = tf.sign(x1)
    mask2 = tf.sign(x2)
    
    x1 = tf.nn.embedding_lookup(self.embedding, x1)
    x2 = tf.nn.embedding_lookup(self.embedding, x2)
    
    x1 = self.inp_dropout(x1, training=training)
    x2 = self.inp_dropout(x2, training=training)
    
    x1, x2 = self.encoder(x1), self.encoder(x2)
    x = []
    
    # attention 1 (multiply)
    a = tf.matmul(x1, x2, transpose_b=True)
    x.append(tf.expand_dims(a, -1))
    
    # attention 2 (add)
    a1 = tf.expand_dims(self.W_1_1(x1), 2)
    a2 = tf.expand_dims(self.W_1_2(x2), 1)
    x.append(self.v_1(tf.tanh(a1 + a2)))
    
    # attention 3 (minus)
    a1 = tf.expand_dims(x1, 2)
    a2 = tf.expand_dims(x2, 1)
    x.append(self.v_2(tf.tanh(self.W_2(tf.abs(a1 - a2)))))
    
    x = tf.concat(x, -1)
    x = self.conv_1(x)
    x = tf.nn.max_pool(x, [1, stride1, stride2, 1], [1, stride1, stride2, 1], 'VALID')
    x = self.conv_2(x)
    x = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
    x = self.conv_3(x)
    x = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
    
    x = self.flatten(x)
    x = self.out_hidden(x, training=training)
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
    'model_dir': '../model/esim',
    'log_path': '../log/esim.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'embedding_path': '../vocab/word.npy',
    'vocab_path': '../vocab/word.txt',
    'batch_size': 32,
    'buffer_size': 200000,
    'hidden_units': 300,
    'fixed_len1': 14,
    'fixed_len2': 8,
    'lr': 4e-4,
    'clip_norm': 10.,
    'num_labels': 3,
    'eval_steps': 550152//32//10,
    'num_patience': 7,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [10]:
params['word2idx'] = get_vocab(params['vocab_path'])
params['vocab_size'] = len(params['word2idx']) + 1

model = Pyramid(params)
model.build([[None, 20], [None, 14]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for ((text1, text2), label) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((text1, text2), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
      loss = tf.reduce_mean(loss)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
    if global_step % params['eval_steps'] == 0:
      # EVALUATION
      m = tf.keras.metrics.Accuracy()

      for ((text1, text2), label) in dataset(is_training=False, params=params):
        logits = model((text1, text2), training=False)
        y_pred = tf.argmax(logits, axis=-1)
        m.update_state(y_true=label, y_pred=y_pred)

      acc = m.result().numpy()
      logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
      history_acc.append(acc)

      if acc > best_acc:
        best_acc = acc
        # you can save model here
      logger.info("Best Accuracy: {:.3f}".format(best_acc))

      if len(history_acc) > params['num_patience'] and is_descending(history_acc):
        logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
        break

W0808 02:33:40.584271 139796796041088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('encoder/forward_lstm/kernel:0', TensorShape([300, 1200])),
 ('encoder/forward_lstm/recurrent_kernel:0', TensorShape([300, 1200])),
 ('encoder/forward_lstm/bias:0', TensorShape([1200])),
 ('encoder/backward_lstm/kernel:0', TensorShape([300, 1200])),
 ('encoder/backward_lstm/recurrent_kernel:0', TensorShape([300, 1200])),
 ('encoder/backward_lstm/bias:0', TensorShape([1200])),
 ('conv2d/kernel:0', TensorShape([7, 7, 3, 32])),
 ('conv2d/bias:0', TensorShape([32])),
 ('conv2d_1/kernel:0', TensorShape([5, 5, 32, 64])),
 ('conv2d_1/bias:0', TensorShape([64])),
 ('conv2d_2/kernel:0', TensorShape([3, 3, 64, 128])),
 ('conv2d_2/bias:0', TensorShape([128])),
 ('dense/kernel:0', TensorShape([600, 300])),
 ('dense_1/kernel:0', TensorShape([600, 300])),
 ('dense_2/kernel:0', TensorShape([300, 1])),
 ('dense_3/kernel:0', TensorShape([600, 300])),
 ('dense_4/kernel:0', TensorShape([300, 1])),
 ('out_hidden/dense_5/kernel:0', TensorShape([768, 300])),
 ('out_hidden/dense_5/bias:0', TensorShape([300

W0808 02:34:05.327376 139796796041088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/clip_ops.py:286: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0808 02:34:05.435937 139796796041088 interactiveshell.py:2882] Step 0 | Loss: 1.1674 | Spent: 24.9 secs | LR: 0.000400
I0808 02:34:10.530395 139796796041088 interactiveshell.py:2882] Step 50 | Loss: 1.1106 | Spent: 5.1 secs | LR: 0.000400
I0808 02:34:15.308517 139796796041088 interactiveshell.py:2882] Step 100 | Loss: 1.0799 | Spent: 4.8 secs | LR: 0.000400
I0808 02:34:19.953603 139796796041088 interactiveshell.py:2882] Step 150 | Loss: 1.0141 | Spent: 4.6 secs | LR: 0.000399
I0808 02:34:24.639735 139796796041088 interactiveshell.py:2882] Step 200 | Loss: 0.8350 | Spent: 4.7 secs | LR: 0.000399
I0808 02:34:29.348435 13979679

Reading ../data/test.txt


I0808 02:37:00.721574 139796796041088 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.702
I0808 02:37:00.722591 139796796041088 interactiveshell.py:2882] Best Accuracy: 0.702
I0808 02:37:03.960448 139796796041088 interactiveshell.py:2882] Step 1750 | Loss: 0.8874 | Spent: 15.9 secs | LR: 0.000393
I0808 02:37:08.858494 139796796041088 interactiveshell.py:2882] Step 1800 | Loss: 1.0048 | Spent: 4.9 secs | LR: 0.000393
I0808 02:37:13.965367 139796796041088 interactiveshell.py:2882] Step 1850 | Loss: 0.6963 | Spent: 5.1 secs | LR: 0.000393
I0808 02:37:19.408742 139796796041088 interactiveshell.py:2882] Step 1900 | Loss: 0.6438 | Spent: 5.4 secs | LR: 0.000392
I0808 02:37:24.724691 139796796041088 interactiveshell.py:2882] Step 1950 | Loss: 0.6447 | Spent: 5.3 secs | LR: 0.000392
I0808 02:37:29.684739 139796796041088 interactiveshell.py:2882] Step 2000 | Loss: 0.5885 | Spent: 5.0 secs | LR: 0.000392
I0808 02:37:34.571427 139796796041088 interactiveshell.py:2882] Step 2050 | Loss: 

...

Reading ../data/test.txt


I0808 06:53:20.296903 139796796041088 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.862
I0808 06:53:20.297857 139796796041088 interactiveshell.py:2882] Best Accuracy: 0.866
I0808 06:53:23.884123 139796796041088 interactiveshell.py:2882] Step 128950 | Loss: 0.3735 | Spent: 17.9 secs | LR: 0.000109
I0808 06:53:30.631010 139796796041088 interactiveshell.py:2882] Step 129000 | Loss: 0.2342 | Spent: 6.7 secs | LR: 0.000109
I0808 06:53:37.446532 139796796041088 interactiveshell.py:2882] Step 129050 | Loss: 0.2367 | Spent: 6.8 secs | LR: 0.000109
I0808 06:53:44.252795 139796796041088 interactiveshell.py:2882] Step 129100 | Loss: 0.2169 | Spent: 6.8 secs | LR: 0.000109
I0808 06:53:51.132206 139796796041088 interactiveshell.py:2882] Step 129150 | Loss: 0.3155 | Spent: 6.9 secs | LR: 0.000109
I0808 06:53:57.943119 139796796041088 interactiveshell.py:2882] Step 129200 | Loss: 0.1893 | Spent: 6.8 secs | LR: 0.000109
I0808 06:54:04.686749 139796796041088 interactiveshell.py:2882] Step 1

Reading ../data/test.txt


I0808 06:57:26.391435 139796796041088 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.860
I0808 06:57:26.392378 139796796041088 interactiveshell.py:2882] Best Accuracy: 0.866
I0808 06:57:27.356796 139796796041088 interactiveshell.py:2882] Step 130650 | Loss: 0.2791 | Spent: 17.7 secs | LR: 0.000108


KeyboardInterrupt: ignored